# 🐛 Debugging & Gradient Checking

Le debugging est une **compétence essentielle** en deep learning !

## 🎯 Objectifs

1. **Gradient Checking** : Vérifier que ta backprop est correcte
2. **Détecter les problèmes** : Vanishing/exploding gradients
3. **Visualiser les gradients** pendant l'entraînement
4. **Erreurs courantes** et comment les corriger
5. **Checklist de debugging** complète

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent))

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)

print("✅ Ready to debug!")

## 1️⃣ Gradient Checking : Vérifier la Backprop

### 💡 L'Idée

Comparer le gradient **analytique** (backprop) avec le gradient **numérique** (approximation).

### 📐 Gradient Numérique

$$
\frac{\partial L}{\partial w} \approx \frac{L(w + \epsilon) - L(w - \epsilon)}{2\epsilon}
$$

où $\epsilon$ est très petit (ex: $10^{-7}$)

### ✅ Si backprop est correcte :
$$
\text{difference} = \frac{\|\nabla_{\text{num}} - \nabla_{\text{backprop}}\|}{\|\nabla_{\text{num}}\| + \|\nabla_{\text{backprop}}\|} < 10^{-7}
$$

In [ ]:
def numerical_gradient(f, x, epsilon=1e-7):
    """
    Calcule le gradient numérique
    
    Args:
        f: fonction qui calcule la loss
        x: paramètre (peut être un tableau)
        epsilon: petite valeur pour l'approximation
    
    Returns:
        grad: gradient numérique
    """
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        old_value = x[idx]
        
        # f(x + epsilon)
        x[idx] = old_value + epsilon
        fx_plus = f(x)
        
        # f(x - epsilon)
        x[idx] = old_value - epsilon
        fx_minus = f(x)
        
        # Gradient
        grad[idx] = (fx_plus - fx_minus) / (2 * epsilon)
        
        # Restaurer
        x[idx] = old_value
        it.iternext()
    
    return grad

def gradient_check(grad_analytical, grad_numerical, threshold=1e-7):
    """
    Compare gradient analytique vs numérique
    """
    numerator = np.linalg.norm(grad_analytical - grad_numerical)
    denominator = np.linalg.norm(grad_analytical) + np.linalg.norm(grad_numerical)
    difference = numerator / (denominator + 1e-10)
    
    print("\n🔍 GRADIENT CHECKING")
    print("="*60)
    print(f"\nDifférence: {difference:.2e}")
    print(f"Threshold: {threshold:.2e}")
    
    if difference < threshold:
        print("\n✅ EXCELLENT! Backpropagation est correcte!")
        return True
    elif difference < 1e-5:
        print("\n⚠️ ACCEPTABLE. Backprop probablement correcte.")
        return True
    else:
        print("\n❌ PROBLÈME! Backpropagation a une erreur!")
        print("\n🐛 À vérifier:")
        print("   • Formules des dérivées")
        print("   • Dimensions des matrices")
        print("   • Ordre des opérations")
        return False

print("✅ Fonctions de gradient checking créées")

### 🧪 Test sur un Exemple Simple

In [ ]:
# Exemple simple : f(x) = x^2, donc f'(x) = 2x
def simple_function(x):
    return np.sum(x ** 2)

# Gradient analytique
x = np.array([1.0, 2.0, 3.0])
grad_analytical = 2 * x

# Gradient numérique
grad_numerical = numerical_gradient(simple_function, x.copy())

print("Test sur f(x) = x²")
print(f"\nGradient analytique (2x): {grad_analytical}")
print(f"Gradient numérique:       {grad_numerical}")

gradient_check(grad_analytical, grad_numerical)

### 🧪 Test sur un Petit Réseau

In [ ]:
# Créer un mini réseau pour tester
class TinyNetwork:
    def __init__(self):
        self.W = np.random.randn(3, 2) * 0.01
        self.b = np.zeros((1, 2))
    
    def forward(self, X):
        Z = np.dot(X, self.W) + self.b
        A = 1 / (1 + np.exp(-Z))  # Sigmoid
        return A
    
    def loss(self, X, Y):
        A = self.forward(X)
        return np.mean((A - Y) ** 2)
    
    def backward(self, X, Y):
        m = X.shape[0]
        
        # Forward
        Z = np.dot(X, self.W) + self.b
        A = 1 / (1 + np.exp(-Z))
        
        # Backward
        dA = 2 * (A - Y) / m
        dZ = dA * A * (1 - A)  # Dérivée sigmoid
        dW = np.dot(X.T, dZ)
        db = np.sum(dZ, axis=0, keepdims=True)
        
        return dW, db

# Données de test
X_test = np.random.randn(5, 3)
Y_test = np.random.randn(5, 2)

net = TinyNetwork()

# Gradient analytique (backprop)
dW_analytical, db_analytical = net.backward(X_test, Y_test)

# Gradient numérique pour W
def loss_w(W):
    net.W = W.reshape(net.W.shape)
    return net.loss(X_test, Y_test)

W_flat = net.W.flatten()
dW_numerical_flat = numerical_gradient(loss_w, W_flat.copy())
dW_numerical = dW_numerical_flat.reshape(net.W.shape)

print("\n🧪 TEST SUR UN PETIT RÉSEAU")
print("="*60)
print("\nGradient par rapport à W:")
result = gradient_check(dW_analytical, dW_numerical)

if result:
    print("\n🎉 La backpropagation fonctionne correctement!")

## 2️⃣ Problèmes Courants : Vanishing/Exploding Gradients

### 🌊 Vanishing Gradient

**Symptôme** : Les gradients deviennent très petits (~0)

**Causes** :
- Activation sigmoid/tanh (gradients < 1)
- Réseau très profond
- Mauvaise initialisation

**Solutions** :
- ✅ Utiliser ReLU au lieu de sigmoid
- ✅ Batch normalization
- ✅ Skip connections (ResNet)
- ✅ He initialization

### 💥 Exploding Gradient

**Symptôme** : Les gradients deviennent très grands (→ ∞)

**Causes** :
- Learning rate trop élevé
- Poids mal initialisés
- RNN sans précautions

**Solutions** :
- ✅ Gradient clipping
- ✅ Learning rate plus petit
- ✅ Bonne initialisation
- ✅ Batch normalization

In [ ]:
def simulate_gradient_flow():
    """
    Simule le flow des gradients dans un réseau profond
    """
    n_layers = 20
    
    # Scénario 1: Sigmoid (vanishing)
    gradients_sigmoid = [1.0]
    for _ in range(n_layers):
        # Gradient de sigmoid ≈ 0.25 en moyenne
        gradients_sigmoid.append(gradients_sigmoid[-1] * 0.25)
    
    # Scénario 2: Mauvaise init (exploding)
    gradients_exploding = [1.0]
    for _ in range(n_layers):
        # Poids trop grands
        gradients_exploding.append(gradients_exploding[-1] * 1.5)
    
    # Scénario 3: ReLU + He init (bon)
    gradients_good = [1.0]
    for _ in range(n_layers):
        # Bien équilibré
        gradients_good.append(gradients_good[-1] * 0.95)
    
    # Visualiser
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('🌊 Gradient Flow dans un Réseau Profond', fontsize=18, fontweight='bold')
    
    layers = range(n_layers + 1)
    
    # Vanishing
    axes[0].semilogy(layers, gradients_sigmoid, 'o-', linewidth=3, markersize=6, color='#e74c3c')
    axes[0].set_title('❌ Vanishing Gradient\n(Sigmoid)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Couche', fontsize=12)
    axes[0].set_ylabel('Magnitude du Gradient (log)', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    axes[0].axhline(y=1e-10, color='red', linestyle='--', label='Trop petit!')
    axes[0].legend()
    
    # Exploding
    axes[1].semilogy(layers, gradients_exploding, 's-', linewidth=3, markersize=6, color='#f39c12')
    axes[1].set_title('❌ Exploding Gradient\n(Mauvaise Init)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Couche', fontsize=12)
    axes[1].set_ylabel('Magnitude du Gradient (log)', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    axes[1].axhline(y=1e10, color='red', linestyle='--', label='Trop grand!')
    axes[1].legend()
    
    # Bon
    axes[2].semilogy(layers, gradients_good, '^-', linewidth=3, markersize=6, color='#2ecc71')
    axes[2].set_title('✅ Gradient Stable\n(ReLU + He Init)', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Couche', fontsize=12)
    axes[2].set_ylabel('Magnitude du Gradient (log)', fontsize=12)
    axes[2].grid(True, alpha=0.3)
    axes[2].axhline(y=1.0, color='green', linestyle='--', label='Stable!')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Observations:")
    print(f"   Vanishing: {gradients_sigmoid[-1]:.2e} (trop petit!)")
    print(f"   Exploding: {gradients_exploding[-1]:.2e} (trop grand!)")
    print(f"   Stable: {gradients_good[-1]:.2e} (juste bien!)")

simulate_gradient_flow()

## 3️⃣ Checklist de Debugging

### 🔍 Avant l'Entraînement

- [ ] **Overfit sur un petit batch** (2-10 exemples)
  - Si ça marche pas → problème d'implémentation
  - Si ça marche → problème de régularisation/données

- [ ] **Vérifier les shapes** de tous les tenseurs
  ```python
  print(f"X: {X.shape}, W: {W.shape}, output: {output.shape}")
  ```

- [ ] **Gradient checking** sur un mini réseau

- [ ] **Initialisation** appropriée (He pour ReLU)

- [ ] **Learning rate** raisonnable (commencer petit: 0.001)

### 🔍 Pendant l'Entraînement

- [ ] **Loss diminue** régulièrement
  - Si stagne → LR trop petit ou problème d'optimisation
  - Si explose → LR trop grand

- [ ] **Train/Val accuracy** augmentent
  - Si train élevé mais val faible → overfitting
  - Si les deux faibles → underfitting

- [ ] **Magnitude des gradients** raisonnable (1e-5 à 1e-1)

- [ ] **Pas de NaN/Inf** dans les activations/gradients

### 🐛 Problèmes Fréquents

| Symptôme | Cause Probable | Solution |
|----------|---------------|----------|
| Loss = NaN | Learning rate trop élevé | Réduire LR |
| Loss ne bouge pas | LR trop petit ou mauvaise init | Augmenter LR ou réinit |
| Loss oscille | Batch size trop petit | Augmenter batch size |
| Accuracy aléatoire | Forward ou backprop incorrecte | Debug étape par étape |
| Train OK, val mauvais | Overfitting | Régularisation, dropout |
| Les deux mauvais | Underfitting ou bug | Plus de capacité ou debug |

## 4️⃣ Outils de Debugging Pratiques

In [ ]:
class DebugNetwork:
    """
    Réseau avec outils de debugging intégrés
    """
    def __init__(self):
        self.activations = {}
        self.gradients = {}
        self.has_nan = False
    
    def check_nan_inf(self, tensor, name):
        """Vérifie NaN/Inf"""
        if np.any(np.isnan(tensor)):
            print(f"❌ NaN détecté dans {name}!")
            self.has_nan = True
        if np.any(np.isinf(tensor)):
            print(f"❌ Inf détecté dans {name}!")
            self.has_nan = True
    
    def check_gradient_magnitude(self, grad, name, threshold_low=1e-8, threshold_high=1e2):
        """Vérifie la magnitude des gradients"""
        magnitude = np.linalg.norm(grad)
        
        if magnitude < threshold_low:
            print(f"⚠️ Gradient de {name} trop petit: {magnitude:.2e} (vanishing?)")
        elif magnitude > threshold_high:
            print(f"⚠️ Gradient de {name} trop grand: {magnitude:.2e} (exploding?)")
        else:
            print(f"✅ Gradient de {name}: {magnitude:.2e} (OK)")
    
    def summary(self):
        """Affiche un résumé de debug"""
        print("\n" + "="*60)
        print("🔍 DEBUG SUMMARY")
        print("="*60)
        
        print("\n📊 Activations:")
        for name, act in self.activations.items():
            print(f"   {name}: shape={act.shape}, mean={act.mean():.4f}, std={act.std():.4f}")
        
        print("\n📊 Gradients:")
        for name, grad in self.gradients.items():
            print(f"   {name}: shape={grad.shape}, norm={np.linalg.norm(grad):.2e}")
        
        if not self.has_nan:
            print("\n✅ Pas de NaN/Inf détecté")

# Exemple d'utilisation
debug_net = DebugNetwork()

# Simuler quelques tenseurs
debug_net.activations['layer1'] = np.random.randn(10, 128)
debug_net.gradients['dW1'] = np.random.randn(784, 128) * 0.01

debug_net.check_nan_inf(debug_net.activations['layer1'], 'layer1')
debug_net.check_gradient_magnitude(debug_net.gradients['dW1'], 'dW1')
debug_net.summary()

print("\n✅ Outils de debugging créés!")

## 🎯 Récapitulatif

### ✅ Ce que tu as appris

1. **Gradient Checking**
   - Compare backprop vs approximation numérique
   - Essentiel pour vérifier ton implémentation
   - Différence < 1e-7 = excellent

2. **Problèmes de Gradients**
   - Vanishing: gradients → 0 (sigmoid, réseaux profonds)
   - Exploding: gradients → ∞ (mauvaise init, LR trop grand)
   - Solutions: ReLU, He init, BatchNorm, gradient clipping

3. **Checklist de Debugging**
   - Overfit sur petit batch d'abord
   - Vérifier shapes et NaN/Inf
   - Monitorer loss et accuracy
   - Magnitude des gradients

4. **Outils Pratiques**
   - Fonctions de vérification automatiques
   - Logging des activations et gradients
   - Debug summary

### 💡 Points clés

- 🐛 **Debug tôt** : Gradient checking dès le début
- 📊 **Monitor** : Loss, accuracy, gradients
- 🔍 **Overfit d'abord** : Si ça marche pas sur 10 exemples, ça marchera pas sur 60k
- ⚠️ **Signes d'alerte** : NaN, loss qui explose, gradients extrêmes

### 🚀 Prochaines Étapes

Maintenant que tu sais debugger :
- Interpréter ce que le réseau a appris
- Visualiser les features
- Comprendre les erreurs

---

**Le debugging est une compétence qui te sauvera des heures de frustration ! 🎓**